In [6]:
import os
import json
import pandas as pd
import traceback

In [7]:
from langchain.chat_models import ChatOpenAI


In [8]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [9]:
KEY=os.getenv("OPENAI_API_KEY")

In [10]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.5)

d:\Python\Lecture6\IntelliMCQ\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [11]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2


In [12]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


In [13]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [14]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"], #varaibles user will pass
    template=TEMPLATE
    )
#Input prompt

In [15]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

#creating a chain: Connection between LLM and prompt

In [16]:
#Prompt for quiz evaluaion

TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [17]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [18]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [19]:
#Now connect both chain using sequential chain

In [20]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [21]:
#now we need a data on the top of which we will create a quiz
#make file data.txt and read the data in file

In [22]:
file_path = os.getenv("FILE_PATH")

In [24]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [25]:
print(TEXT)

Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other scientists, bio

In [26]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [27]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"


In [28]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

d:\Python\Lecture6\IntelliMCQ\env\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [29]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:1913
Prompt Tokens:1192
Completion Tokens:721
Total Cost:0.00323


In [30]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [31]:
quiz=response.get("quiz")

In [32]:
quiz=json.loads(quiz)

In [33]:
quiz

{'1': {'mcq': 'What is the scientific study of life called?',
  'options': {'a': 'Geology',
   'b': 'Biology',
   'c': 'Chemistry',
   'd': 'Physics'},
  'correct': 'b'},
 '2': {'mcq': 'Which theme in biology explains the unity and diversity of life?',
  'options': {'a': 'Evolution',
   'b': 'Photosynthesis',
   'c': 'Mitosis',
   'd': 'Osmosis'},
  'correct': 'a'},
 '3': {'mcq': 'What is the basic unit of life that processes hereditary information?',
  'options': {'a': 'Atoms', 'b': 'Molecules', 'c': 'Cells', 'd': 'Tissues'},
  'correct': 'c'},
 '4': {'mcq': 'What is important to life as it allows organisms to move, grow, and reproduce?',
  'options': {'a': 'Energy processing',
   'b': 'Photosynthesis',
   'c': 'Cell division',
   'd': 'Oxygen production'},
  'correct': 'a'},
 '5': {'mcq': 'What emerged on Earth more than 3.7 billion years ago?',
  'options': {'a': 'Humans', 'b': 'Dinosaurs', 'c': 'Life', 'd': 'Plants'},
  'correct': 'c'}}

In [34]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [35]:
quiz_table_data

[{'MCQ': 'What is the scientific study of life called?',
  'Choices': 'a: Geology | b: Biology | c: Chemistry | d: Physics',
  'Correct': 'b'},
 {'MCQ': 'Which theme in biology explains the unity and diversity of life?',
  'Choices': 'a: Evolution | b: Photosynthesis | c: Mitosis | d: Osmosis',
  'Correct': 'a'},
 {'MCQ': 'What is the basic unit of life that processes hereditary information?',
  'Choices': 'a: Atoms | b: Molecules | c: Cells | d: Tissues',
  'Correct': 'c'},
 {'MCQ': 'What is important to life as it allows organisms to move, grow, and reproduce?',
  'Choices': 'a: Energy processing | b: Photosynthesis | c: Cell division | d: Oxygen production',
  'Correct': 'a'},
 {'MCQ': 'What emerged on Earth more than 3.7 billion years ago?',
  'Choices': 'a: Humans | b: Dinosaurs | c: Life | d: Plants',
  'Correct': 'c'}]

In [36]:
quiz=pd.DataFrame(quiz_table_data)

In [37]:
quiz

,MCQ,Choices,Correct
0,What is the scientific study of life called?,a: Geology | b: Biology | c: Chemistry | d: Ph...,b
1,Which theme in biology explains the unity and ...,a: Evolution | b: Photosynthesis | c: Mitosis ...,a
2,What is the basic unit of life that processes ...,a: Atoms | b: Molecules | c: Cells | d: Tissues,c
3,What is important to life as it allows organis...,a: Energy processing | b: Photosynthesis | c: ...,a
4,What emerged on Earth more than 3.7 billion ye...,a: Humans | b: Dinosaurs | c: Life | d: Plants,c


In [38]:
quiz.to_csv("quiz.csv",index=False)
#creates a new .csv file with the same name

In [1]:
import logging
import os
from datetime import datetime


LOG_FILE=f"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [2]:
LOG_FILE

'04_24_2024_22_16_50.log'